# Envío de eventos a S3

Este cuaderno es muy sencillo, solo se trata de código para enviar eventos como archivos de JSON separados a S3 basados en las interacciones de los usuarios para Amazon Personalize.

## Configuración

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

Asegúrese de actualizar los ARN de la campaña y el conjunto de datos que figura a continuación.

In [ ]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Campaign ARN
Campaign_ARN = "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-demo-camp"

# Dataset Group Arn:
datasetGroupArn = "arn:aws:personalize:us-east-1:059124553121:dataset-group/personalize-lambda-demo"

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')


# S3 Configuration
bucket = "personalizedemo9000chris"

## Simulación de eventos

Este código dependerá de 3 usuarios que sabemos que existen en conjunto de datos de MovieLens:

1. 196
2. 13
3. 198

Se seleccionaron de forma aleatoria y ahora seleccionaremos 3 películas de forma aleatoria para que el usuario interactúe:

1. 225
2. 203
3. 242

Cada uno de los usuarios interactuará con cada película y cada interacción se escribirá en S3 como un archivo JSON diferente.

In [ ]:
users = ["196", "13", "198"]
items = ["225", "203", "242"]

In [ ]:
def write_interaction_to_JSON(user, item):
    # An event needs a user, session, time, type, and a properties dictionary.
    time_of_interaction = int(time.time())
    # This is set for simplification
    session_id = user
    user_id = user
    event_type = "watched"
    properties = {
        'itemId': str(item)
    }
    properties = json.dumps(properties)
    
    event_headers = {
        'userId': user,
        'sessionId': session_id
    }
    
    event_content = {
        'sentAt': time_of_interaction,
        'eventType': event_type,
        'properties': properties
    }
    event_list = [event_content]
    json_struct = [event_headers, event_content]
    json_filename = str(time_of_interaction) + ".json"
    
    with open(json_filename, 'w') as outfile:
        json.dump(json_struct, outfile)
    
    return json_filename

def send_interactions_to_s3():
    for user in users:
        for item in items:
            filename = write_interaction_to_JSON(user, item)
            boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)
            print(filename)
            # Sleep needed to make sure everyone gets a nice new timestamp
            time.sleep(2)
            

In [ ]:
send_interactions_to_s3()

In [ ]:
# The code below is just to explore what to put in our Lambda:

with open("1568076414.json", 'r') as input_file:
    content = json.loads(input_file.read())
    # assume tracking ID was provided via an environment variable and replace
    trackingId = "a68baf70-f0f2-4c66-af2d-e11047a0372f"
    print(content[0]['userId'])
    
    personalize_events.put_events(
        trackingId = trackingId,
        userId = content[0]['userId'],
        sessionId = content[0]['sessionId'],
        eventList = [content[1]]
    )